In [2]:
import graphlab
import os
from graphlab import mxnet as mx
#Limitacion de Hilos
graphlab.set_runtime_config('GRAPHLAB_DEFAULT_NUM_PYLAMBDA_WORKERS', 4)
#Descarga y guardar el modelo iter45
pretrained_model_iter_45=graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
pretrained_model_iter_45.save('imagenet_model_iter45')
#Carga inception_1k_v3 (https://github.com/dmlc/mxnet-model-gallery/blob/master/imagenet-1k-inception-v3.md)
pretrained_model_inceptionv3=mx.pretrained_model.load_path('F:\\MSc Misis\\4 Semestre\\Tesis 1\\Modelos\\imagenet1k_inception_v3')
#Carga inception_21k (https://github.com/dmlc/mxnet-model-gallery/blob/master/imagenet-21k-inception.md)
pretrained_model_inception21k=mx.pretrained_model.load_path('F:\\MSc Misis\\4 Semestre\\Tesis 1\\Modelos\\imagenet21k_inception_bn')

[INFO] graphlab.mxnet.base: CUDA support is currently not available on this platform. GPU support is disabled.


This non-commercial license of GraphLab Create for academic use is assigned to ma-varon@uniandes.edu.co and will expire on November 18, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\IngMario\AppData\Local\Temp\graphlab_server_1496103282.log.0


Downloading http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45/dir_archive.ini to C:/Users/IngMario/AppData/Local/Temp/graphlab-IngMario/2572/aa80bd80-9746-4508-a7e2-a6e5b56a2f62.ini

Downloading http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45/objects.bin to C:/Users/IngMario/AppData/Local/Temp/graphlab-IngMario/2572/7b668288-106d-4a52-aa3e-1ad6c4c31526.bin

In [28]:

#creación de SArray para introducir columna de label en Training set
def Label_Training_column_set(num_images_Class1,num_label_Class1):
    a=[]
    b=[num_label_Class1]
    i=0
    if(i<num_images_Class1):
        while (i<num_images_Class1):
            a.extend(b)
            i=i+1
    label_array=graphlab.SArray(data=a, dtype=int)
    return label_array



#Función Preprocesamiento del Set de Datos de Entrenamiento
def Preproc_set(path, number_images_Class, num_label_Class):
    Path_images=path 
    image_sframe = graphlab.image_analysis.load_images(Path_images, "jpg", with_path=True,recursive=True)
    image_sframe['resized']=graphlab.image_analysis.resize(image_sframe['image'], 200, 200, 1)
    label_array=Label_Training_column_set(number_images_Class,num_label_Class )
    image_sframe.add_column(label_array, name='label')
    return image_sframe
              
#Carga de los datos
TS1= Preproc_set("C:\\Users\\IngMario\\Documents\\Data\\Untitled Folder\\DS_Casco",40,1)
TS2= Preproc_set("C:\\Users\\IngMario\\Documents\\Data\\Untitled Folder\\DS_Sin_Casco",37,0)
TS1=TS1.append(TS2)

Test_d1= Preproc_set("C:\\Users\\IngMario\\Documents\\Data\\Untitled Folder\\DS_Test_Casco",20,1)
Test_d2= Preproc_set("C:\\Users\\IngMario\\Documents\\Data\\Untitled Folder\\DS_Test_Sin_Casco",14,0)
Test_d1=Test_d1.append(Test_d2)



In [4]:
#Carga de modelos
pretrained_model_iter_45=graphlab.load_model('imagenet_model_iter45')

In [29]:
#Extracción de Caracerísticas de DataSet de Entrenamiento y Prueba con modelos iter_45, inceptionv3, inception21k
TS1['features_iter_45']= pretrained_model_iter_45.extract_features(TS1[['image']])
TS1['features_inceptionv3']= pretrained_model_inceptionv3.extract_features(TS1[['image']])
TS1['features_inception21K']= pretrained_model_inception21k.extract_features(TS1[['image']])
Test_d1['features_iter_45']= pretrained_model_iter_45.extract_features(Test_d1[['image']])
Test_d1['features_inceptionv3']= pretrained_model_inceptionv3.extract_features(Test_d1[['image']])
Test_d1['features_inception21K']= pretrained_model_inception21k.extract_features(Test_d1[['image']])
TS1.save('TS.sframe')
Test_d1.save('Test_d.sframe')

Images being resized.

[INFO] graphlab.mxnet.pretrained_model: Detect image shape mismatches network input shape. Perform resize to shape (299, 299, 3)
[INFO] graphlab.mxnet.pretrained_model: Detect image shape mismatches network input shape. Perform resize to shape (224, 224, 3)


Images being resized.

[INFO] graphlab.mxnet.pretrained_model: Detect image shape mismatches network input shape. Perform resize to shape (299, 299, 3)
[INFO] graphlab.mxnet.pretrained_model: Detect image shape mismatches network input shape. Perform resize to shape (224, 224, 3)


In [30]:
training_data, validation_data = TS1.random_split(0.9)
helmet_classifier_model_NN = graphlab.neuralnet_classifier.create(training_data, target='label',  features=['features_iter_45', 
                                                                                                            'features_inceptionv3',
                                                                                                           'features_inception21K'],
                                                                validation_set=validation_data)
helmet_classifier_model_DT = graphlab.decision_tree_classifier.create(training_data, target='label', features=['features_iter_45', 
                                                                                                            'features_inceptionv3',
                                                                                                           'features_inception21K'],
                                                                   validation_set=validation_data)
helmet_classifier_model_BTC=graphlab.boosted_trees_classifier.create(training_data, target='label', features=['features_iter_45', 
                                                                                                            'features_inceptionv3',
                                                                                                           'features_inception21K'],
                                                                   max_iterations=10, validation_set=validation_data,
                                                                  max_depth=5, step_size=0.5, metric='accuracy', 
                                                                  early_stopping_rounds=3)
helmet_classifier_model_SVM= graphlab.svm_classifier.create(training_data, target='label', features=['features_iter_45',
                                                                                                    'features_inceptionv3',
                                                                                                    'features_inception21K'],
                                                                   solver='lbfgs',max_iterations=8,validation_set=validation_data)

Using network:

### network layers ###
layer[0]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 10
layer[1]: SigmoidLayer
layer[2]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 2
layer[3]: SoftmaxLayer
### end network layers ###

### network parameters ###
learning_rate = 0.001
momentum = 0.9
### end network parameters ###



Creating neuralnet using cpu

Training with batch size = 66

+-----------+----------+--------------+-------------------+---------------------+-----------------+

| Iteration | Examples | Elapsed Time | Training-accuracy | Validation-accuracy | Examples/second |

...

| 5         | 12       | 1.000000  | 0.753059     | 0.969697          | 0.909091            |

| 6         | 14       | 1.000000  | 0.807599     | 1.000000          | 0.909091            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [32]:
results_NN=helmet_classifier_model_NN.evaluate(Test_d1)
results_DT=helmet_classifier_model_DT.evaluate(Test_d1)
results_BTC=helmet_classifier_model_BTC.evaluate(Test_d1)
results_SVM=helmet_classifier_model_SVM.evaluate(Test_d1)

results_SVM

{'accuracy': 0.8529411764705882, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        1        |   18  |
 |      1       |        0        |   2   |
 |      0       |        1        |   3   |
 |      0       |        0        |   11  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns], 'f1_score': 0.8780487804878048, 'precision': 0.8571428571428571, 'recall': 0.9}